In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

In [2]:
!pip install autopep8
!pip install jupyter_contrib_nbextensions==0.5.1
!pip install bokeh==0.12.15

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

from dwave.system.composites import EmbeddingComposite
import dwavebinarycsp as dbc

In [4]:
P = 77  

bP = "{:010b}".format(P)    # "{:010b}" formats for 10-bit binary
print(bP)

0001001101


In [5]:
csp = dbc.factories.multiplication_circuit(5)

print(next(iter(csp.constraints)))

Constraint.from_configurations(frozenset({(1, 0, 0), (1, 1, 1), (0, 1, 0), (0, 0, 0)}), ('a0', 'b0', 'p0'), Vartype.BINARY, name='AND(a0, b0) = p0')


In [6]:
# Convert CSP to BQM 

bqm = dbc.stitch(csp, min_classical_gap=.1)

print("BQM has {} variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 75 variables: 
	['a0', 'b0', 'p0', 'b1', 'and0,1', 'b2', 'and0,2', 'b3', 'and0,3', 'b4', 'and0,4', 'a1', 'and1,0', 'p1', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'sum1,2', 'carry1,2', 'and1,3', 'sum1,3', 'carry1,3', 'and1,4', 'a2', 'and2,0', 'p2', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'sum2,2', 'carry2,2', 'and2,3', 'sum2,3', 'carry2,3', 'and2,4', 'a3', 'and3,0', 'p3', 'carry3,0', 'and3,1', 'sum3,1', 'carry3,1', 'and3,2', 'sum3,2', 'carry3,2', 'and3,3', 'sum3,3', 'carry3,3', 'and3,4', 'a4', 'and4,0', 'p4', 'carry4,0', 'and4,1', 'sum4,1', 'carry4,1', 'and4,2', 'sum4,2', 'carry4,2', 'and4,3', 'sum4,3', 'carry4,3', 'and4,4', 'p5', 'carry5,0', 'p6', 'carry5,1', 'p7', 'carry5,2', 'p8', 'p9']


In [7]:
p_vars = ['p0', 'p1', 'p2', 'p3','p4','p5', 'p6', 'p7','p8', 'p9']
#p_vars = ['p0', 'p1', 'p2', 'p3','p4','p5', 'p6', 'p7']

# Convert P from decimal to binary
fixed_variables = dict(zip(reversed(p_vars), "{:010b}".format(P)))
fixed_variables = {var: int(x) for(var, x) in fixed_variables.items()}

# Fix product variables
for var, value in fixed_variables.items():
    bqm.fix_variable(var, value)
    
print("BQM has {} non-fixed variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 65 non-fixed variables: 
	['a0', 'b0', 'b1', 'and0,1', 'b2', 'and0,2', 'b3', 'and0,3', 'b4', 'and0,4', 'a1', 'and1,0', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'sum1,2', 'carry1,2', 'and1,3', 'sum1,3', 'carry1,3', 'and1,4', 'a2', 'and2,0', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'sum2,2', 'carry2,2', 'and2,3', 'sum2,3', 'carry2,3', 'and2,4', 'a3', 'and3,0', 'carry3,0', 'and3,1', 'sum3,1', 'carry3,1', 'and3,2', 'sum3,2', 'carry3,2', 'and3,3', 'sum3,3', 'carry3,3', 'and3,4', 'a4', 'and4,0', 'carry4,0', 'and4,1', 'sum4,1', 'carry4,1', 'and4,2', 'sum4,2', 'carry4,2', 'and4,3', 'sum4,3', 'carry4,3', 'and4,4', 'carry5,0', 'carry5,1', 'carry5,2']


# Solving on D-Wave

In [9]:
sampler = BraketDWaveSampler(device_arn='arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6')

from helpers.embedding import embeddings
from dwave.system import FixedEmbeddingComposite

#sampler_fixed_embedding = FixedEmbeddingComposite(sampler)
sampler = EmbeddingComposite(sampler)

print("Mapping from BQM variables to qubits for the Dwave system:\n\n\t{}", sampler)


Mapping from BQM variables to qubits for the Dwave system:

	{} <dwave.system.composites.embedding.EmbeddingComposite object at 0x7f8b97e9b9d0>


In [10]:
import time
start_time = time.time()
#sampleset = sampler_fixed_embedding.sample(bqm, num_reads=100)
sampleset = sampler.sample(bqm, num_reads=100)

print("--- %s seconds ---" % (time.time() - start_time))

print("Best solution found: \n",sampleset.first.sample)

--- 11.283055305480957 seconds ---
Best solution found: 
 {'a0': 1, 'a1': 0, 'a2': 0, 'a3': 1, 'a4': 1, 'and0,1': 0, 'and0,2': 1, 'and0,3': 1, 'and0,4': 0, 'and1,0': 0, 'and1,1': 0, 'and1,2': 0, 'and1,3': 0, 'and1,4': 0, 'and2,0': 0, 'and2,1': 0, 'and2,2': 0, 'and2,3': 0, 'and2,4': 0, 'and3,0': 1, 'and3,1': 0, 'and3,2': 1, 'and3,3': 1, 'and3,4': 0, 'and4,0': 1, 'and4,1': 0, 'and4,2': 1, 'and4,3': 1, 'and4,4': 0, 'b0': 1, 'b1': 0, 'b2': 1, 'b3': 1, 'b4': 0, 'carry1,0': 0, 'carry1,1': 0, 'carry1,2': 0, 'carry1,3': 0, 'carry2,0': 0, 'carry2,1': 1, 'carry2,2': 0, 'carry2,3': 0, 'carry3,0': 0, 'carry3,1': 0, 'carry3,2': 0, 'carry3,3': 0, 'carry4,0': 1, 'carry4,1': 0, 'carry4,2': 1, 'carry4,3': 0, 'carry5,0': 1, 'carry5,1': 0, 'carry5,2': 1, 'sum1,1': 1, 'sum1,2': 1, 'sum1,3': 0, 'sum2,1': 0, 'sum2,2': 0, 'sum2,3': 0, 'sum3,1': 1, 'sum3,2': 1, 'sum3,3': 1, 'sum4,1': 1, 'sum4,2': 0, 'sum4,3': 1}


In [11]:
from helpers.convert import to_base_ten
a, b = to_base_ten(sampleset.first.sample, 5)

print("Given integer P={}, found factors a={} and b={}".format(P, a, b))

Given integer P=77, found factors a=25 and b=13


In [12]:
from helpers import draw

from collections import OrderedDict

def response_to_dict(sampleset):
    results_dict = OrderedDict()
    for sample, energy in sampleset.data(['sample', 'energy']):
        a, b = to_base_ten(sample, 5)
        if (a, b) not in results_dict:
            results_dict[(a, b)] = round(energy, 2)
            
    return results_dict

results = response_to_dict(sampleset)

draw.energy_of(results)

Loading BokehJS ...

In [13]:
from collections import OrderedDict

def response_to_dict(sampleset):
    results_dict = OrderedDict()
    for sample, energy in sampleset.data(['sample', 'energy']):
        a, b = to_base_ten(sample, 4)
        if (a, b) not in results_dict:
            if a * b == P:
                results_dict[(a, b)] = round(energy, 2)
            
    return results_dict

results = response_to_dict(sampleset)

draw.energy_of(results)

In [31]:
print("Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost()} USD")

Task Summary
{'arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6': {'shots': 100, 'tasks': {'COMPLETED': 1}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 0.3190000000 USD
